In [1]:
import pathlib

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tifffile as tf  # write tiff files
from PIL import Image  # read tiff files
from toml import load
from tqdm import tqdm  # progress bar

In [2]:
CELL_TYPE = "PBMC"

In [3]:
path = pathlib.Path(
    "../../../8.cytopick_analysis/results/PBMC/single_cell_predictions.parquet"
).resolve(strict=True)

# read in the data
df = pd.read_parquet(path)
# show all columns
pd.set_option("display.max_columns", None)
df.head()

,apoptosis_prob,control_prob,pyroptosis_prob,label_true,label_pred,data_split,shuffle,class_name,index,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,Metadata_inducer1_concentration_unit,Metadata_inducer2,Metadata_inducer2_concentration,Metadata_inducer2_concentration_unit,Metadata_ImageNumber,Metadata_Plate,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Nuclei_Number_Object_Number,Metadata_Treatment,Metadata_Dose,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cytoplasm_AreaShape_BoundingBoxMaximum_X,Metadata_Cytoplasm_AreaShape_BoundingBoxMaximum_Y,Metadata_Cytoplasm_AreaShape_BoundingBoxMinimum_X,Metadata_Cytoplasm_AreaShape_BoundingBoxMinimum_Y,Metadata_Site,labels,correct,image_path,image_crop_path
0,0.127945,0.345422,0.526633,2,2,train,True,pyroptosis,2021413.0,PBMC,E09,53760.0,6.0,DMSO,0.025,%,LPS,10.000,µg_per_ml,None,nan,None,298.0,*70117_20230210MM1_Gasdermin514_CP_BC430856,2876.0,2876.0,2882.0,2882.0,LPS,10.000_µg_per_ml,1662.832335,1548.467066,1674.0,1563.0,1654.0,1535.0,10.0,pyroptosis,True,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...
1,0.155791,0.333143,0.511066,2,2,train,True,pyroptosis,4926511.0,PBMC,K02,54477.0,6.0,DMSO,0.025,%,LPS,1.000,µg_per_ml,Nigericin,1.000,µM,191.0,*70117_20230210MM1_Gasdermin514_CP_BC430856,1246.0,1246.0,1249.0,1249.0,LPS_Nigericin,1.000_µg_per_ml_1.000_µM,1079.899642,596.154122,1090.0,606.0,1071.0,586.0,15.0,pyroptosis,True,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...
2,0.126370,0.376186,0.497444,2,2,train,True,pyroptosis,2467561.0,PBMC,G02,51149.0,6.0,Disulfiram,1.000,µM,LPS,10.000,µg_per_ml,None,nan,None,191.0,*70117_20230210MM1_Gasdermin514_CP_BC430856,292.0,292.0,292.0,292.0,LPS,10.000_µg_per_ml,50.653266,160.055276,68.0,172.0,42.0,137.0,15.0,pyroptosis,True,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...
3,0.118203,0.393621,0.488176,2,2,train,True,pyroptosis,746227.0,PBMC,C08,55156.0,6.0,DMSO,0.025,%,LPS,0.100,µg_per_ml,None,nan,None,282.0,*70117_20230210MM1_Gasdermin514_CP_BC430856,3695.0,3695.0,3700.0,3700.0,LPS,0.100_µg_per_ml,2026.294833,1939.197568,2037.0,1951.0,2016.0,1929.0,10.0,pyroptosis,True,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...
4,0.120190,0.398426,0.481384,2,2,train,True,pyroptosis,7184062.0,PBMC,N02,47664.0,6.0,Disulfiram,1.000,µM,LPS,1.000,µg_per_ml,Nigericin,10.000,µM,3.0,*70117_20230210MM1_Gasdermin514_CP_BC430856,2180.0,2180.0,2181.0,2181.0,LPS_Nigericin,1.000_µg_per_ml_10.000_µM,1036.483221,1301.399329,1047.0,1311.0,1027.0,1292.0,3.0,pyroptosis,True,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...


In [4]:
# loop through each image and assign a scale bar
for image_path in df["image_crop_path"]:
    print(image_path)
    # open the image
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    # image to array
    image_array = np.array(image)
    resolution = 3.3446  # pixels per micrometer
    # get the image size
    image_size = image_array.shape
    if image_array.shape[1] <= 100:
        scale_bar_length = 5  # um
        scale_bar_height = 1  # pixels
        padding = 3  # pixels
    elif image_array.shape[1] > 100:
        scale_bar_length = 100
        scale_bar_height = 10
        padding = 10
    # get the bottom right most corner based on scale 1 % of pixels
    scale_bar_x = image_size[1] - (scale_bar_length * resolution) - padding - padding
    scale_bar_y = image_size[0] - (scale_bar_height) - padding
    print(scale_bar_x, scale_bar_y)
    # draw the scale bar
    new = cv2.rectangle(
        image_array,
        (int(scale_bar_x), int(scale_bar_y)),
        (image_size[1] - padding, image_size[0] - padding),
        (255, 255, 255),
        -1,
    )
    # save the image with the scale bar via cv2
    cv2.imwrite(image_path, new)
#

/home/lippincm/Documents/4TB/data/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_train_df_im1_im3_im4_composite_image_crop_cell_0.png
77.277 96
/home/lippincm/Documents/4TB/data/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_train_df_im1_im3_im4_composite_image_crop_cell_1.png
77.277 96
/home/lippincm/Documents/4TB/data/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_train_df_im1_im3_im4_composite_image_crop_cell_2.png
77.277 96
/home/lippincm/Documents/4TB/data/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_train_df_im1_im3_im4_composite_image_crop_cell_3.png
77.277 96
/home/lippincm/Documents/4TB/data/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_train_df_im1_im3_im4_composite_image_crop_cell_4.png
77.277 96
/home/lippincm/Documents/4TB/data/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_train_df_im1_im3_im4_composite_